In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

!pip install llama-index-embeddings-huggingface

!pip install llama-index

!pip install llama-index-llms-huggingface

In [22]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
import time

start = time.time()

print(llm.complete("Write a simple Python script to print all even numbers between 0 and 10, inclusive").text)

time.time() - start

In [1]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

/home/drmathew/.local/lib/python3.9/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
LLAMA2_7B_CHAT = "meta-llama/Llama-2-7b-chat-hf"

In [3]:
selected_model = LLAMA2_7B_CHAT

In [4]:
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
import time

start = time.time()

print(llm.complete("In Python, make a function that prints all even numbers between 0 and 10 inclusive.").text)

print(f"Time elapsed: {time.time() - start}")

Of course! Here is a function in Python that prints all even numbers between 0 and 10 inclusive:

def print_even_numbers(nums):
    for num in nums:
        if num % 2 == 0:
            print(num)

# Test the function
print_even_numbers([0, 2, 4, 6, 8, 10])

I hope this helps! Let me know if you have any other questions.
Time elapsed: 15.672654628753662


In [19]:
set_global_tokenizer(
    AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf").encode
)

In [20]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
document = SimpleDirectoryReader(input_files=["data/" + 'essay.txt']).load_data()
index = VectorStoreIndex.from_documents(document, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm, streaming=True)

In [31]:
start = time.time()
r = query_engine.query("What did the author do growing up?")
r.print_response_stream()
print()
print(time.time() - start)

Based on the given context, the author grew up writing and programming outside of school. They wrote short stories and tried to write programs on the IBM 1401 computer in 9th grade, but they couldn't figure out what to do with it. Later, with the advent of microcomputers, the author began writing simple games, a program to predict how high model rockets would fly, and a word processor that their father used to write at least one book. The author also studied philosophy in college, but found it boring, and eventually switched to AI.19.30453372001648


In [93]:
import requests
import json
from bs4 import BeautifulSoup
import os

In [33]:
API_TOKEN = "rxfMXxfsdsKGb8urL8shecBbBp37ZBuvU5zo39sm"
API_ENDPOINT = f"https://api.marketaux.com/v1/news/all"

payload = {
    "api_token": API_TOKEN,
    "entity_types": "equity,index,etf", # should include industries parameter
    "industries": "Technology",
    "countries": "us",
    "published_after": "2024-05-10T15:00",
    "domains": "finance.yahoo.com"
}

In [ ]:
payload = {
    "api_token": API_TOKEN,
    "entity_types": "equity,index,etf", # should include industries parameter
#     "industries": "Technology",
    "countries": "us",
    "published_after": "2024-05-10T13:00",
    "published_before": "2024-05-10T14:00",
    "domains": "finance.yahoo.com"
}

In [54]:
def get_urls(payload):
    r = requests.get("https://api.marketaux.com/v1/news/all", params=payload)
    output = r.text
    json_output = json.loads(output)
    data = json_output['data']
    
    urls = []
    
    for i in range(len(data)):
        url = data[i]['url']
        if url.startswith("https://finance.yahoo.com/news"):
            urls.append(url)
        
    return urls

In [56]:
def get_article_content(url):
    r = requests.get(url)
    
    if r.status_code != 200:
        return 'Invalid Response'
    
    soup = BeautifulSoup(r.content)
    body = soup.find('div', attrs={'class': 'caas-body'})
    content = ""
    
    for text in body.find_all('p'):
        content += text.text
    
    return content

In [58]:
def write_to_file(filename, content):
    with open(, 'w') as file:
        file.write(content)

In [95]:
def create_documents():
    payload = {
        "api_token": API_TOKEN,
        "entity_types": "equity,index,etf", # should include industries parameter
    #     "industries": "Technology",
        "countries": "us",
#         "published_after": "2024-05-10T13:00",
#         "published_before": "2024-05-10T14:00",
        "domains": "finance.yahoo.com"
    }
    
    for add_day in range(5):
        for add_hour in range(7):
            start = "2024-05-" + str(10 + add_day) + "T" + str(13 + add_hour) + ":00"
            finish = "2024-05-" + str(10 + add_day) + "T" + str(13 + add_hour + 1) + ":00"
            payload["published_after"] = start
            payload["published_before"] = finish
            
            urls = get_urls(payload)
            for i in range(len(urls)):
                print(urls[i])
                content = get_article_content(urls[i])
                if content != "Invalid Response": 
                    filename = os.path.join("articles", "article" + str(add_day) + str(add_hour) + str(i) + ".txt")
                    write_to_file(filename, content)
                else:
                    print("Above URL Skipped")

In [59]:
create_documents()

https://finance.yahoo.com/news/2-beaten-down-stocks-buy-134500970.html
https://finance.yahoo.com/news/stock-market-today-stocks-rise-dow-eyes-8th-straight-win-as-rate-cut-hopes-abound-133123854.html
https://finance.yahoo.com/news/3-rock-solid-dividend-stocks-133000363.html
https://finance.yahoo.com/news/tsmcs-record-sales-surge-whats-144109570.html
https://finance.yahoo.com/news/groupon-grpn-q1-earnings-revenues-143100508.html?.tsrc=rss
https://finance.yahoo.com/news/cathie-wood-goes-bargain-hunting-154500669.html
https://finance.yahoo.com/news/mcdonald-readies-5-meal-deal-160738126.html
https://finance.yahoo.com/news/q1-2024-applied-optoelectronics-inc-163026968.html?.tsrc=rss
Above URL Skipped
https://finance.yahoo.com/news/openai-announce-chatgpt-product-updates-175757460.html?.tsrc=rss
https://finance.yahoo.com/news/want-500-dividend-income-heres-175637541.html
https://finance.yahoo.com/news/stock-market-today-dow-secures-eighth-straight-winning-session-183420207.html
https://finan

In [44]:
r = requests.get("https://finance.yahoo.com/video/apple-catching-ai-revolution-dan-142925403.html?.tsrc=rss")
s = BeautifulSoup(r.content)
body = s.find('div', attrs={'class': 'caas-body'})
body

In [53]:
type(r.status_code)

int

In [47]:
url = "https://finance.yahoo.com/news/q1-2024-applied-optoelectronics-inc-163026968.html?.tsrc=rss"
r = requests.get(url)
soup = BeautifulSoup(r.content)
body = soup.find('div')

In [48]:
r.reason

'Not Found'

In [60]:
yf_documents = SimpleDirectoryReader(input_dir="articles").load_data()
yf_index = VectorStoreIndex.from_documents(yf_documents, embed_model=embed_model)
yf_query_engine = yf_index.as_query_engine(llm=llm, streaming=True)

In [77]:
stocks = yf_query_engine.query("Based on the ingested articles, return the best stocks to invest in and give reasons why.")
stocks.print_response_stream()

Based on the articles provided, the following are the best stocks to invest in and why:

1. Bristol Myers Squibb (NYSE: BMY) - Bristol Myers has a strong pipeline of new medicines, including a subcutaneous version of 

KeyboardInterrupt: 

In [78]:
response_2 = yf_query_engine.query("Based on the ingested articles, return the 5 best stocks to invest in and give reasons why.")
response_2.print_response_stream()

Based on the information provided in the articles, here are five stocks that could be considered as potential investments:

1. Vertex Pharmaceuticals (NASDAQ: VRTX): Vertex has a strong track record of developing and commercializing innovative drugs for rare and underserved diseases, such as cystic fibrosis. The company's pipeline is also promising, with multiple new product launches planned before the end of the decade. Its strong balance sheet and market leadership in cystic fibrosis therapies provide a solid foundation for long-term growth.
2. Bristol Myers Squibb (NYSE: BMY): Bristol Myers has a diversified portfolio of pharmaceutical products, including cancer drugs, and is investing in new medicines with potential blockbuster status. The company's recent patent cliffs are a concern, but it has a plan to address these issues and is developing a subcutaneous version of its Opdivo drug to expand its indications. Bristol Myers' dividend program is also attractive, with a forward yiel

In [79]:
response_3 = yf_query_engine.query("Based on the ingested articles, return a set of stocks that would best diversify a portfolio with the best returns.")
response_3.print_response_stream()

Based on the articles provided, here are three stocks that could potentially diversify a portfolio and provide good returns:

1. CRISPR Therapeutics (CRSP): This biotechnology company is at the forefront of gene editing and has the potential to revolutionize the treatment of various diseases. According to The Motley Fool's Stock Advisor analyst team, CRISPR Therapeutics could produce monster returns in the coming years.
2. Ulta Beauty (ULTA): As a leading beauty retailer, Ulta Beauty has a strong track record of growth and has been identified as a top stock for investors to buy now. The company has a diverse portfolio of brands and offers a wide range of products, making it well-positioned for continued success.
3. e.l.f. Beauty (ELF): This cosmetics company has a strong presence in the growing e-commerce market and has been identified as a top stock for investors looking to diversify their portfolio. e.l.f. Beauty has a strong brand and has been successful in expanding its product lin

In [90]:
new_prompt = "Return the financial securities that are most mentioned across the articles in a positive sentiment." + \
"The securities returned can be stocks or ETFs. Ensure that the securities returned are of different industries " + \
"that can be put together to make a diversified portfolio. List the stock and the industry associated with it" + \
"and double check that a majority of the industries aren't the same."

response_3 = yf_query_engine.query(new_prompt)
response_3.print_response_stream()

Based on the provided context information, the following financial securities are mentioned positively across the articles:

1. Stock: NVIDIA (NVDA) - Industry: Technology
2. Stock: Advanced Micro Devices (AMD) - Industry: Technology
3. Stock: Micron Technology (MU) - Industry: Technology
4. ETF: iShares Semiconductor ETF (SOXX) - Industry: Technology
5. Stock: Qualcomm (QCOM) - Industry: Technology
6. Stock: Intel (INTC) - Industry: Technology
7. Stock: Taiwan Semiconductor Manufacturing (TSM) - Industry: Technology
8. Stock: Broadcom (AVGO) - Industry: Technology
9. Stock: Intel (INTC) - Industry: Technology
10. ETF: Vanguard S&P 500 ETF (VOO) - Industry: Finance

These securities are from different industries, making up a diversified portfolio. Note that the performance of these securities is not guaranteed, and it's essential to conduct thorough research and consult with a financial advisor before making any investment decisions.

In [100]:
new_prompt = "Gather the financial securities that are most mentioned across the articles in a positive sentiment." + \
"The securities gathered can be stocks or ETFs. Return a list of 12 securities in which no more than half of the " + \
"securities are from the same industry. Create this list so that they " + \
"can be put together to make a diversified portfolio. List the stock and the industry associated with it" + \
"and double check that a majority of the industries aren't the same."

response_3 = yf_query_engine.query(new_prompt)
response_3.print_response_stream()

Based on the provided articles, the following are the financial securities that are most mentioned in a positive sentiment:

1. Nvidia (NVDA) - Technology
2. Advanced Micro Devices (AMD) - Technology
3. Intel (INTC) - Technology
4. Micron Technology (MU) - Technology
5. Qualcomm (QCOM) - Technology
6. Broadcom (AVGO) - Technology
7. Texas Instruments (TXN) - Technology
8. Vanguard S&P 500 ETF (VOO) - Finance
9. iShares Trust - iShares Semiconductor ETF (SOXX) - Technology
10. SPDR S&P 500 ETF Trust (SPY) - Finance
11. iShares Trust - iShares NASDAQ Biotechnology ETF (IBB) - Healthcare
12. iShares Trust - iShares MSCI ACWI ETF (ACWI) - Finance

Note: The industries listed are based on the classification of the securities by the articles and may not align with their exact industry classification.

This list includes a mix of technology, healthcare, and finance sectors, which can be used to create a diversified portfolio. It's important to note that investing in a single security or indus

In [101]:
response_3.response_txt

"Based on the provided articles, the following are the financial securities that are most mentioned in a positive sentiment:\n\n1. Nvidia (NVDA) - Technology\n2. Advanced Micro Devices (AMD) - Technology\n3. Intel (INTC) - Technology\n4. Micron Technology (MU) - Technology\n5. Qualcomm (QCOM) - Technology\n6. Broadcom (AVGO) - Technology\n7. Texas Instruments (TXN) - Technology\n8. Vanguard S&P 500 ETF (VOO) - Finance\n9. iShares Trust - iShares Semiconductor ETF (SOXX) - Technology\n10. SPDR S&P 500 ETF Trust (SPY) - Finance\n11. iShares Trust - iShares NASDAQ Biotechnology ETF (IBB) - Healthcare\n12. iShares Trust - iShares MSCI ACWI ETF (ACWI) - Finance\n\nNote: The industries listed are based on the classification of the securities by the articles and may not align with their exact industry classification.\n\nThis list includes a mix of technology, healthcare, and finance sectors, which can be used to create a diversified portfolio. It's important to note that investing in a single

In [124]:
new_prompt_risk_averse = "Gather the financial securities that are most mentioned across the articles in a positive sentiment." + \
"The securities gathered can be stocks or ETFs. Return a list of 12 securities in which no more than half of the " + \
"securities are from the same industry. This list of securities should also be tailored to a very risk averse investor who " + \
"is more focused on stability instead of rapid growth. " + \
"Create this list so that they " + \
"can be put together to make a diversified portfolio. List the stock and the industry associated with it" + \
"and double check that a majority of the industries aren't the same."

response_risk= yf_query_engine.query(new_prompt_risk_averse)
response_risk.print_response_stream()

Based on the provided articles, the following are the financial securities that are most mentioned in a positive sentiment:

1. Vanguard S&P 500 ETF (VOO) - Industry: Financials
2. iShares Trust - iShares Semiconductor ETF (SOXX) - Industry: Technology
3. SPDR S&P 500 ETF Trust (SPY) - Industry: Financials
4. Micron Technology, Inc. (MU) - Industry: Technology
5. Nvidia Corporation (NVDA) - Industry: Technology
6. Qualcomm Incorporated (QCOM) - Industry: Technology
7. Taiwan Semiconductor Manufacturing Company Limited (TSM) - Industry: Technology
8. Advanced Micro Devices, Inc. (AMD) - Industry: Technology
9. Intel Corporation (INTC) - Industry: Technology
10. Broadcom Inc. (AVGO) - Industry: Technology
11. Cisco Systems, Inc. (CSCO) - Industry: Technology
12. Procter & Gamble Company (PG) - Industry: Consumer Goods

Notes:

* No more than half of the securities are from the same industry.
* The securities are tailored to a very risk-averse investor who is more focused on stability ins

In [122]:
prompts_dict = yf_query_engine.get_prompts()
print(list(prompts_dict.keys()))

['response_synthesizer:text_qa_template', 'response_synthesizer:refine_template']


In [108]:
import re

pattern = r'\(([^)]+)\)'
matches = re.findall(pattern, response_3.response_txt)

In [109]:
matches

['NVDA',
 'AMD',
 'INTC',
 'MU',
 'QCOM',
 'AVGO',
 'TXN',
 'VOO',
 'SOXX',
 'SPY',
 'IBB',
 'ACWI']

In [117]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing both the new"
    " context and your own knowledge, update or repeat the existing answer.\n"
)
refine_template = PromptTemplate(refine_template_str)

In [111]:
text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\nUsing"
    " both the context information and also using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you can also"
    " answer the question on your own.\n"
)

In [116]:
print(text_qa_template_str)

Context information is below.
---------------------
{context_str}
---------------------
Using both the context information and also using your own knowledge, answer the question: {query_str}
If the context isn't helpful, you can also answer the question on your own.



In [120]:
# should test adding system prompt and incorporating user preferences like risk aversion and industry preferences